 adversarial attacks and defenses in image classification using Reinforcement Learning (RL), the task could be defined as follows:

Task: Given a list of image classification attacks (e.g., FGSM, C&W, etc.) and a list of defenses, the goal is to determine the optimal attack and defense strategy using RL.

The action space could be the set of all possible attacks and defenses. The state space could be the current attack and defense being used, and the observation space could be the outcome of the attack (success or failure) and the performance of the defense.

In this context, the RL agent’s goal is to learn an optimal policy that selects the best attack and defense strategy based on the current state and observations. The agent receives a reward when it successfully identifies the optimal strategy.

According to recent research, the attacker can derive optimal attacks by planning in polynomial time or learning with polynomial sample complexity using standard RL techniques1. An ensemble-learning-based adversarial attack (ELAA) targeting image-classification models, which aggregate and optimize multiple RL base learners, has been proposed2. Experimental results show that the attack success rate for the ensemble model is about 35% higher than for a single model2.

For defenses, adversarial training-based defense methods have shown to increase model performance for medical image classification tasks on adversarial examples3. The optimal defense policy for the victim can be computed as the solution to a stochastic Stackelberg game1.

Please note that the effectiveness of attacks and defenses can vary depending on the specific image classification task and the characteristics of the data. Therefore, it’s important to evaluate different strategies in the context of your specific task.

```python
import numpy as np
from typing import List, Tuple

class ImageClassificationEnv:
    def __init__(self, attacks: List[str], defenses: List[str]):
        self.attacks = attacks
        self.defenses = defenses
        self.current_attack = None
        self.current_defense = None
        
    def reset(self) -> Tuple[int, int]:
        self.current_attack = np.random.choice(len(self.attacks))
        self.current_defense = np.random.choice(len(self.defenses))
        return self.current_attack, self.current_defense
    
    def step(self, action: Tuple[int, int]) -> Tuple[Tuple[int, int], float, bool]:
        attack, defense = action
        success = self.evaluate_attack(attack, defense)
        reward = 1.0 if success else -1.0
        done = True
        self.current_attack = attack
        self.current_defense = defense
        return (attack, defense), reward, done
    
    def evaluate_attack(self, attack: int, defense: int) -> bool:
        # Implement the logic to evaluate the success of the attack against the defense
        # Return True if the attack is successful, False otherwise
        pass

class RLAgent:
    def __init__(self, num_attacks: int, num_defenses: int, learning_rate: float, discount_factor: float):
        self.num_attacks = num_attacks
        self.num_defenses = num_defenses
        self.learning_rate = learning_rate
        self.discount_factor = discount_factor
        self.q_table = np.zeros((num_attacks, num_defenses))
        
    def choose_action(self, state: Tuple[int, int], epsilon: float) -> Tuple[int, int]:
        if np.random.uniform(0, 1) < epsilon:
            attack = np.random.choice(self.num_attacks)
            defense = np.random.choice(self.num_defenses)
        else:
            attack, defense = np.unravel_index(np.argmax(self.q_table, axis=None), self.q_table.shape)
        return attack, defense
    
    def update_q_table(self, state: Tuple[int, int], action: Tuple[int, int], reward: float, next_state: Tuple[int, int]):
        current_q = self.q_table[state]
        next_max_q = np.max(self.q_table[next_state])
        new_q = (1 - self.learning_rate) * current_q + self.learning_rate * (reward + self.discount_factor * next_max_q)
        self.q_table[state] = new_q

def train_agent(env: ImageClassificationEnv, agent: RLAgent, num_episodes: int, epsilon: float):
    for episode in range(num_episodes):
        state = env.reset()
        done = False
        while not done:
            action = agent.choose_action(state, epsilon)
            next_state, reward, done = env.step(action)
            agent.update_q_table(state, action, reward, next_state)
            state = next_state

# Example usage
attacks = ["FGSM", "C&W", "PGD"]
defenses = ["Adversarial Training", "Randomized Smoothing", "Ensemble Defense"]
env = ImageClassificationEnv(attacks, defenses)
agent = RLAgent(len(attacks), len(defenses), learning_rate=0.1, discount_factor=0.9)
train_agent(env, agent, num_episodes=1000, epsilon=0.1)
```

```python 
import gym
from gym import spaces
import numpy as np
from stable_baselines3 import PPO
from stable_baselines3.common.env_checker import check_env
from transformers import AutoModelForImageClassification, AutoFeatureExtractor
import torch
from typing import List, Tuple

class AdversarialEnv(gym.Env):
    def __init__(self, attacks: List[str], defenses: List[str], image_classifier: str):
        super(AdversarialEnv, self).__init__()
        self.attacks = attacks
        self.defenses = defenses
        self.image_classifier = image_classifier
        self.model = AutoModelForImageClassification.from_pretrained(image_classifier)
        self.feature_extractor = AutoFeatureExtractor.from_pretrained(image_classifier)
        self.action_space = spaces.Discrete(len(attacks) * len(defenses))
        self.observation_space = spaces.Box(low=0, high=1, shape=(2,), dtype=np.float32)

    def reset(self):
        self.attack_idx = np.random.randint(0, len(self.attacks))
        self.defense_idx = np.random.randint(0, len(self.defenses))
        observation = np.array([self.attack_idx, self.defense_idx], dtype=np.float32)
        return observation

    def step(self, action):
        attack_idx, defense_idx = divmod(action, len(self.defenses))
        success, performance = self.perform_action(attack_idx, defense_idx)
        observation = np.array([attack_idx, defense_idx], dtype=np.float32)
        reward = 1 if success else -1
        done = True
        info = {'performance': performance}
        return observation, reward, done, info

    def perform_action(self, attack_idx, defense_idx):
        # Implement the logic for performing the selected attack and defense
        # Return the success flag and performance metric based on the outcome
        pass

# Example usage
attacks = ["FGSM", "C&W", "PGD"]
defenses = ["Adversarial Training", "Randomized Smoothing", "Ensemble Defense"]
image_classifier = "google/vit-base-patch16-224"

env = AdversarialEnv(attacks, defenses, image_classifier)
check_env(env)

model = PPO("MlpPolicy", env, verbose=1)
model.learn(total_timesteps=10000)

# Evaluate the trained agent
obs = env.reset()
attack_idx, defense_idx = env.attack_idx, env.defense_idx
action, _ = model.predict(obs, deterministic=True)
success, performance = env.perform_action(attack_idx, defense_idx)
print(f"Selected defense for attack {attacks[attack_idx]}: {defenses[defense_idx]}")
print(f"Attack success: {success}, Performance: {performance}")


```

```python 
import numpy as np
import torch
from typing import Tuple

def perform_action(self, attack_idx: int, defense_idx: int) -> Tuple[bool, float]:
    # Load a sample image (replace with your own logic)
    image = self.load_image("sample_image.jpg")

    # Apply the selected attack
    attacked_image = self.apply_attack(image, self.attacks[attack_idx])

    # Apply the selected defense
    defended_image = self.apply_defense(attacked_image, self.defenses[defense_idx])

    # Perform image classification using the loaded model
    inputs = self.feature_extractor(images=defended_image, return_tensors="pt")
    with torch.no_grad():
        logits = self.model(**inputs).logits
    predicted_label = logits.argmax(-1).item()

    # Evaluate the success of the attack and defense
    success = predicted_label != self.get_true_label(image)
    performance = self.evaluate_performance(image, defended_image)

    return success, performance

def load_image(self, image_path: str) -> np.ndarray:
    # Implement the logic to load and preprocess the image
    pass

def apply_attack(self, image: np.ndarray, attack: str) -> np.ndarray:
    # Implement the logic to apply the selected attack to the image
    pass

def apply_defense(self, image: np.ndarray, defense: str) -> np.ndarray:
    # Implement the logic to apply the selected defense to the image
    pass

def get_true_label(self, image: np.ndarray) -> int:
    # Implement the logic to get the true label of the image
    pass

def evaluate_performance(self, original_image: np.ndarray, defended_image: np.ndarray) -> float:
    # Implement the logic to evaluate the performance of the defense
    pass

```

In [3]:
from typing import List
import plotly.graph_objects as go
from collections import Counter

def plot_string_lists_analysis(*string_lists: List[str]) -> None:
    """
    Plots the frequency distribution of strings in multiple lists using Plotly.

    Args:
        *string_lists: Variable number of lists containing strings.

    Returns:
        None
    """
    fig = go.Figure()

    for i, string_list in enumerate(string_lists, start=1):
        # Count the frequency of each string in the list
        string_counts = Counter(string_list)

        # Extract the strings and their corresponding frequencies
        strings, frequencies = zip(*string_counts.items())

        # Create a bar trace for each string list
        trace = go.Bar(
            x=strings,
            y=frequencies,
            name=f"List {i}",
            text=frequencies,
            textposition="auto",
        )
        fig.add_trace(trace)

    # Customize the layout
    fig.update_layout(
        title="Frequency Distribution of Strings",
        xaxis_title="Strings",
        yaxis_title="Frequency",
        barmode="group",
        height=500,
    )

    # Display the plot
    fig.show()


# Example usage
list1 = ["apple", "banana", "apple", "orange", "banana", "apple"]
list2 = ["dog", "cat", "dog", "bird", "cat", "dog", "dog"]
list3 = ["red", "blue", "green", "red", "blue", "red", "green", "red"]

plot_string_lists_analysis(list1, list2, list3)


In [4]:
from typing import Dict, List
import plotly.graph_objects as go

def plot_nested_string_lists(data: Dict[str, List[List[str]]]) -> None:
    """
    Plot a dictionary where the values are lists of lists of strings.

    Args:
        data: A dictionary where the keys are labels and the values are lists of lists of strings.
    """
    fig = go.Figure()

    for label, nested_lists in data.items():
        for i, string_list in enumerate(nested_lists, start=1):
            lengths = [len(s) for s in string_list]
            fig.add_trace(go.Box(y=lengths, name=f"{label} - List {i}"))

    fig.update_layout(
        title="Nested String Length Distribution",
        xaxis_title="Label - List",
        yaxis_title="String Length",
    )

    fig.show()

# Example usage
data = {
    "Group A": [
        ["apple", "banana", "cherry"],
        ["dog", "cat", "elephant", "lion"],
    ],
    "Group B": [
        ["python", "java", "c++"],
        ["car", "bike", "train", "airplane", "ship"],
    ],
}

plot_nested_string_lists(data)


In [5]:
from typing import Dict, List
import plotly.graph_objects as go

def plot_nested_string_lists(data: Dict[str, List[List[str]]]) -> None:
    """
    Plots a dictionary of lists of lists of strings using Plotly.

    Args:
        data: A dictionary where the keys are the names of the categories,
              and the values are lists of lists of strings.

    Returns:
        None
    """
    fig = go.Figure()

    for category, nested_lists in data.items():
        for i, string_list in enumerate(nested_lists, start=1):
            # Convert the list of strings to a string
            string_data = ", ".join(string_list)

            # Create a scatter trace for each string list
            trace = go.Scatter(
                x=[i],
                y=[category],
                mode="markers",
                marker=dict(size=10),
                text=string_data,
                hoverinfo="text",
                name=category,
            )
            fig.add_trace(trace)

    # Customize the layout
    fig.update_layout(
        title="Visualization of Dictionary of Lists of Lists of Strings",
        xaxis_title="Index",
        yaxis_title="Category",
        height=500,
    )

    # Display the plot
    fig.show()


# Example usage
data = {
    "Category 1": [
        ["apple", "banana", "orange"],
        ["dog", "cat", "bird"],
    ],
    "Category 2": [
        ["red", "blue", "green"],
        ["car", "bike", "train"],
    ],
    "Category 3": [
        ["soccer", "basketball", "tennis"],
        ["pizza", "burger", "pasta"],
    ],
}

plot_nested_string_lists(data)


In [6]:
from typing import List, Dict, Tuple
import plotly.graph_objects as go
from plotly.subplots import make_subplots

def plot_data_structures(
    data1: Dict[str, List[List[str]]],
    data2: Dict[str, List[List[str]]],
    data3: List[List[str]],
    data4: List[str]
) -> None:
    """
    Plots various data structures using Plotly.

    Args:
        data1: A dictionary where the keys are the names of the categories,
               and the values are lists of lists of strings.
        data2: A dictionary where the keys are the names of the categories,
               and the values are lists of lists of strings.
        data3: A list of lists of strings.
        data4: A list of strings.

    Returns:
        None
    """
    # Create subplots
    fig = make_subplots(rows=2, cols=2, subplot_titles=("Data 1", "Data 2", "Data 3", "Data 4"))

    # Plot data1
    for category, nested_lists in data1.items():
        for i, string_list in enumerate(nested_lists, start=1):
            string_data = ", ".join(string_list)
            trace = go.Scatter(
                x=[i],
                y=[category],
                mode="markers",
                marker=dict(size=10),
                text=string_data,
                hoverinfo="text",
                name=category,
            )
            fig.add_trace(trace, row=1, col=1)

    # Plot data2
    for category, nested_lists in data2.items():
        for i, string_list in enumerate(nested_lists, start=1):
            string_data = ", ".join(string_list)
            trace = go.Bar(
                x=[category],
                y=[len(string_list)],
                text=string_data,
                textposition="auto",
                name=category,
            )
            fig.add_trace(trace, row=1, col=2)

    # Plot data3
    for i, string_list in enumerate(data3, start=1):
        string_data = ", ".join(string_list)
        trace = go.Scatter(
            x=[i],
            y=[len(string_list)],
            mode="lines+markers",
            text=string_data,
            hoverinfo="text",
            name=f"List {i}",
        )
        fig.add_trace(trace, row=2, col=1)

    # Plot data4
    trace = go.Bar(
        x=data4,
        y=[1] * len(data4),
        text=data4,
        textposition="auto",
        name="Data 4",
    )
    fig.add_trace(trace, row=2, col=2)

    # Customize layout
    fig.update_layout(
        title="Statistical Analysis of Data Structures",
        height=800,
        showlegend=False,
    )

    # Display the plot
    fig.show()


# Example usage
data1 = {
    "Category 1": [
        ["apple", "banana", "orange"],
        ["dog", "cat", "bird"],
    ],
    "Category 2": [
        ["red", "blue", "green"],
        ["car", "bike", "train"],
    ],
}

data2 = {
    "Group A": [
        ["python", "java", "c++"],
        ["html", "css", "javascript"],
    ],
    "Group B": [
        ["math", "physics", "chemistry"],
        ["history", "geography", "literature"],
    ],
}

data3 = [
    ["apple", "banana", "orange"],
    ["dog", "cat", "bird"],
    ["red", "blue", "green"],
]

data4 = ["python", "java", "c++", "javascript"]

plot_data_structures(data1, data2, data3, data4)


In [7]:
from typing import Dict, List, Tuple
import plotly.graph_objects as go

def plot_string_data(data: Dict[str, List[List[str]]],
                     nested_data: Dict[str, List[List[str]]],
                     list_of_lists: List[List[str]],
                     string_list: List[str]) -> None:
    """
    Plot various data structures containing strings using Plotly.

    Args:
        data: A dictionary where the keys are labels and the values are lists of lists of strings.
        nested_data: A dictionary where the keys are labels and the values are lists of lists of strings.
        list_of_lists: A list of lists of strings.
        string_list: A list of strings.
    """
    fig = go.Figure()

    # Plot data from the dictionary
    for label, nested_lists in data.items():
        for i, string_list in enumerate(nested_lists, start=1):
            lengths = [len(s) for s in string_list]
            fig.add_trace(go.Box(y=lengths, name=f"{label} - List {i}"))

    # Plot data from the nested dictionary
    for label, nested_lists in nested_data.items():
        for i, string_list in enumerate(nested_lists, start=1):
            lengths = [len(s) for s in string_list]
            fig.add_trace(go.Box(y=lengths, name=f"Nested {label} - List {i}"))

    # Plot data from the list of lists
    for i, string_list in enumerate(list_of_lists, start=1):
        lengths = [len(s) for s in string_list]
        fig.add_trace(go.Box(y=lengths, name=f"List of Lists - List {i}"))

    # Plot data from the string list
    lengths = [len(s) for s in string_list]
    fig.add_trace(go.Box(y=lengths, name="String List"))

    fig.update_layout(
        title="String Length Distribution",
        xaxis_title="Data Structure",
        yaxis_title="String Length",
    )

    fig.show()

# Example usage
data = {
    "Group A": [
        ["apple", "banana", "cherry"],
        ["dog", "cat", "elephant", "lion"],
    ],
    "Group B": [
        ["python", "java", "c++"],
        ["car", "bike", "train", "airplane", "ship"],
    ],
}

nested_data = {
    "Category X": [
        ["red", "blue", "green"],
        ["circle", "square", "triangle"],
    ],
    "Category Y": [
        ["small", "medium", "large"],
        ["fast", "slow"],
    ],
}

list_of_lists = [
    ["apple", "banana", "orange"],
    ["cat", "dog", "rabbit", "hamster"],
    ["car", "bicycle", "motorcycle"],
]

string_list = ["python", "programming", "data", "analysis"]

plot_string_data(data, nested_data, list_of_lists, string_list)


In [11]:
from typing import List, Dict, Union
import plotly.graph_objects as go
from plotly.subplots import make_subplots

def plot_data_structures(
    *data_structures: Union[Dict[str, List[List[str]]], List[List[str]], List[str]]
) -> None:
    """
    Plots various data structures using Plotly based on user input.

    Args:
        *data_structures: Variable number of data structures, which can be:
            - A dictionary where the keys are the names of the categories,
              and the values are lists of lists of strings.
            - A list of lists of strings.
            - A list of strings.

    Returns:
        None
    """
    num_plots = len(data_structures)
    num_rows = (num_plots + 1) // 2
    num_cols = 2

    # Create subplots
    fig = make_subplots(rows=num_rows, cols=num_cols, subplot_titles=[f"Data {i+1}" for i in range(num_plots)])

    for i, data in enumerate(data_structures, start=1):
        if isinstance(data, dict):
            # Plot dictionary of lists of lists of strings
            for category, nested_lists in data.items():
                for j, string_list in enumerate(nested_lists, start=1):
                    string_data = ", ".join(string_list)
                    trace = go.Scatter(
                        x=[j],
                        y=[category],
                        mode="markers",
                        marker=dict(size=10),
                        text=string_data,
                        hoverinfo="text",
                        name=category,
                    )
                    fig.add_trace(trace, row=(i - 1) // num_cols + 1, col=(i - 1) % num_cols + 1)
        elif isinstance(data, list) and isinstance(data[0], list):
            # Plot list of lists of strings
            for j, string_list in enumerate(data, start=1):
                string_data = ", ".join(string_list)
                trace = go.Scatter(
                    x=[j],
                    y=[len(string_list)],
                    mode="lines+markers",
                    text=string_data,
                    hoverinfo="text",
                    name=f"List {j}",
                )
                fig.add_trace(trace, row=(i - 1) // num_cols + 1, col=(i - 1) % num_cols + 1)
        elif isinstance(data, list) and isinstance(data[0], str):
            # Plot list of strings
            trace = go.Bar(
                x=data,
                y=[1] * len(data),
                text=data,
                textposition="auto",
                name="String List",
            )
            fig.add_trace(trace, row=(i - 1) // num_cols + 1, col=(i - 1) % num_cols + 1)

    # Customize layout
    fig.update_layout(
        title="Statistical Analysis of Data Structures",
        height=num_rows * 400,
        showlegend=False,
    )

    # Display the plot
    fig.show()


# Example usage
data1 = {
    "Category 1": [
        ["apple", "banana", "orange"],
        ["dog", "cat", "bird"],
    ],
    "Category 2": [
        ["red", "blue", "green"],
        ["car", "bike", "train"],
    ],
}

data2 = [
    ["python", "java", "c++"],
    ["html", "css", "javascript"],
]

data3 = ["math", "physics", "chemistry"]

# User can provide any combination of data structures
plot_data_structures(data3)
plot_data_structures(data1, data3)
plot_data_structures(data2, data3)
plot_data_structures(data1, data2, data3)


In [9]:
from typing import Dict, List, Union
import plotly.graph_objects as go

def plot_string_data(*data_structures: Union[Dict[str, List[List[str]]], List[List[str]], List[str]]) -> None:
    """
    Plot various data structures containing strings using Plotly.

    Args:
        *data_structures: Variable number of data structures, which can be:
            - A dictionary where the keys are labels and the values are lists of lists of strings.
            - A list of lists of strings.
            - A list of strings.
    """
    fig = go.Figure()

    for ds_index, data_structure in enumerate(data_structures, start=1):
        if isinstance(data_structure, dict):
            for label, nested_lists in data_structure.items():
                for i, string_list in enumerate(nested_lists, start=1):
                    lengths = [len(s) for s in string_list]
                    fig.add_trace(go.Box(y=lengths, name=f"Dict {ds_index} - {label} - List {i}"))
        elif isinstance(data_structure, list) and isinstance(data_structure[0], list):
            for i, string_list in enumerate(data_structure, start=1):
                lengths = [len(s) for s in string_list]
                fig.add_trace(go.Box(y=lengths, name=f"List of Lists {ds_index} - List {i}"))
        elif isinstance(data_structure, list) and isinstance(data_structure[0], str):
            lengths = [len(s) for s in data_structure]
            fig.add_trace(go.Box(y=lengths, name=f"String List {ds_index}"))

    fig.update_layout(
        title="String Length Distribution",
        xaxis_title="Data Structure",
        yaxis_title="String Length",
    )

    fig.show()

# Example usage
data1 = {
    "Group A": [
        ["apple", "banana", "cherry"],
        ["dog", "cat", "elephant", "lion"],
    ],
    "Group B": [
        ["python", "java", "c++"],
        ["car", "bike", "train", "airplane", "ship"],
    ],
}

data2 = [
    ["apple", "banana", "orange"],
    ["cat", "dog", "rabbit", "hamster"],
    ["car", "bicycle", "motorcycle"],
]

data3 = ["python", "programming", "data", "analysis"]

plot_string_data(data1, data2, data3)


In [12]:
from typing import Any, List, Dict, Union
import plotly.graph_objects as go
from plotly.subplots import make_subplots

def plot_data_structures(*data_structures: Any) -> None:
    """
    Plots various data structures using Plotly based on user input.

    Args:
        *data_structures: Variable number of data structures of any type.

    Returns:
        None
    """
    num_plots = len(data_structures)
    num_rows = (num_plots + 1) // 2
    num_cols = 2

    # Create subplots
    fig = make_subplots(rows=num_rows, cols=num_cols, subplot_titles=[f"Data {i+1}" for i in range(num_plots)])

    for i, data in enumerate(data_structures, start=1):
        if isinstance(data, dict):
            # Plot dictionary
            for key, value in data.items():
                if isinstance(value, list):
                    # Plot dictionary of lists
                    for j, item in enumerate(value, start=1):
                        if isinstance(item, list):
                            # Plot dictionary of lists of lists
                            item_data = ", ".join(str(x) for x in item)
                        else:
                            item_data = str(item)
                        trace = go.Scatter(
                            x=[j],
                            y=[key],
                            mode="markers",
                            marker=dict(size=10),
                            text=item_data,
                            hoverinfo="text",
                            name=key,
                        )
                        fig.add_trace(trace, row=(i - 1) // num_cols + 1, col=(i - 1) % num_cols + 1)
                else:
                    # Plot dictionary of non-list values
                    trace = go.Bar(
                        x=[key],
                        y=[value],
                        text=str(value),
                        textposition="auto",
                        name=key,
                    )
                    fig.add_trace(trace, row=(i - 1) // num_cols + 1, col=(i - 1) % num_cols + 1)
        elif isinstance(data, list):
            # Plot list
            for j, item in enumerate(data, start=1):
                if isinstance(item, list):
                    # Plot list of lists
                    item_data = ", ".join(str(x) for x in item)
                    trace = go.Scatter(
                        x=[j],
                        y=[len(item)],
                        mode="lines+markers",
                        text=item_data,
                        hoverinfo="text",
                        name=f"List {j}",
                    )
                else:
                    # Plot list of non-list values
                    trace = go.Bar(
                        x=[j],
                        y=[item],
                        text=str(item),
                        textposition="auto",
                        name=f"Item {j}",
                    )
                fig.add_trace(trace, row=(i - 1) // num_cols + 1, col=(i - 1) % num_cols + 1)
        else:
            # Plot single value
            trace = go.Bar(
                x=["Value"],
                y=[data],
                text=str(data),
                textposition="auto",
                name="Value",
            )
            fig.add_trace(trace, row=(i - 1) // num_cols + 1, col=(i - 1) % num_cols + 1)

    # Customize layout
    fig.update_layout(
        title="Statistical Analysis of Data Structures",
        height=num_rows * 400,
        showlegend=False,
    )

    # Display the plot
    fig.show()


# Example usage
data1 = {
    "Category 1": [
        ["apple", "banana", "orange"],
        ["dog", "cat", "bird"],
    ],
    "Category 2": [
        ["red", "blue", "green"],
        ["car", "bike", "train"],
    ],
}

data2 = [
    ["python", "java", "c++"],
    ["html", "css", "javascript"],
]

data3 = ["math", "physics", "chemistry"]

data4 = {
    "Fruit": "apple",
    "Color": "red",
    "Number": 42,
}

data5 = [1, 2, 3, 4, 5]

data6 = "Hello, World!"

# User can provide any combination and structure of data
plot_data_structures(data1, data2, data3)
plot_data_structures(data4, data5, data6)
plot_data_structures(data1, data4, data5)
plot_data_structures(data2, data3, data6)


In [13]:
from typing import List, Dict, Union, Any
import plotly.graph_objects as go
from plotly.subplots import make_subplots

def plot_data_structures(*data_structures: Any) -> None:
    """
    Plots various data structures using Plotly based on user input.

    Args:
        *data_structures: Variable number of data structures of any type.

    Returns:
        None
    """
    num_plots = len(data_structures)
    num_rows = (num_plots + 1) // 2
    num_cols = 2

    # Create subplots
    fig = make_subplots(rows=num_rows, cols=num_cols, subplot_titles=[f"Data {i+1}" for i in range(num_plots)])

    for i, data in enumerate(data_structures, start=1):
        if isinstance(data, dict):
            # Plot dictionary
            for key, value in data.items():
                trace = go.Scatter(
                    x=[str(key)],
                    y=[str(value)],
                    mode="markers",
                    marker=dict(size=10),
                    text=str(value),
                    hoverinfo="text",
                    name=str(key),
                )
                fig.add_trace(trace, row=(i - 1) // num_cols + 1, col=(i - 1) % num_cols + 1)
        elif isinstance(data, list):
            # Plot list
            for j, item in enumerate(data, start=1):
                trace = go.Scatter(
                    x=[j],
                    y=[str(item)],
                    mode="markers",
                    marker=dict(size=10),
                    text=str(item),
                    hoverinfo="text",
                    name=f"Item {j}",
                )
                fig.add_trace(trace, row=(i - 1) // num_cols + 1, col=(i - 1) % num_cols + 1)
        else:
            # Plot single value
            trace = go.Scatter(
                x=["Value"],
                y=[str(data)],
                mode="markers",
                marker=dict(size=10),
                text=str(data),
                hoverinfo="text",
                name="Value",
            )
            fig.add_trace(trace, row=(i - 1) // num_cols + 1, col=(i - 1) % num_cols + 1)

    # Customize layout
    fig.update_layout(
        title="Statistical Analysis of Data Structures",
        height=num_rows * 400,
        showlegend=False,
    )

    # Display the plot
    fig.show()


# Example usage
data1 = {
    "Category 1": [
        ["apple", "banana", "orange"],
        ["dog", "cat", "bird"],
    ],
    "Category 2": [
        ["red", "blue", "green"],
        ["car", "bike", "train"],
    ],
}

data2 = [
    ["python", "java", "c++"],
    ["html", "css", "javascript"],
]

data3 = ["math", "physics", "chemistry"]

data4 = 42

data5 = {
    "Name": "John",
    "Age": 30,
    "City": "New York"
}

# User can provide any combination of data structures
plot_data_structures(data3)
plot_data_structures(data1, data3)
plot_data_structures(data2, data3)
plot_data_structures(data1, data2, data3)
plot_data_structures(data4)
plot_data_structures(data5)


In [14]:
from typing import Any, List, Dict, Union
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import numpy as np

def plot_data_structures(*data_structures: Any) -> None:
    """
    Plots various data structures using Plotly based on user input.

    Args:
        *data_structures: Variable number of data structures of any type.

    Returns:
        None
    """
    num_plots = len(data_structures)
    num_rows = (num_plots + 1) // 2
    num_cols = 2

    # Create subplots
    fig = make_subplots(rows=num_rows, cols=num_cols, subplot_titles=[f"Data {i+1}" for i in range(num_plots)])

    for i, data in enumerate(data_structures, start=1):
        if isinstance(data, dict):
            # Plot dictionary
            for key, value in data.items():
                if isinstance(value, list) and isinstance(value[0], np.ndarray):
                    # Plot dictionary of lists of tensors
                    for j, tensor in enumerate(value, start=1):
                        trace = go.Heatmap(
                            z=tensor,
                            colorscale="Viridis",
                            name=f"{key} - Tensor {j}",
                        )
                        fig.add_trace(trace, row=(i - 1) // num_cols + 1, col=(i - 1) % num_cols + 1)
                elif isinstance(value, list):
                    # Plot dictionary of lists
                    for j, item in enumerate(value, start=1):
                        if isinstance(item, list):
                            # Plot dictionary of lists of lists
                            item_data = ", ".join(str(x) for x in item)
                        else:
                            item_data = str(item)
                        trace = go.Scatter(
                            x=[j],
                            y=[key],
                            mode="markers",
                            marker=dict(size=10),
                            text=item_data,
                            hoverinfo="text",
                            name=key,
                        )
                        fig.add_trace(trace, row=(i - 1) // num_cols + 1, col=(i - 1) % num_cols + 1)
                else:
                    # Plot dictionary of non-list values
                    trace = go.Bar(
                        x=[key],
                        y=[value],
                        text=str(value),
                        textposition="auto",
                        name=key,
                    )
                    fig.add_trace(trace, row=(i - 1) // num_cols + 1, col=(i - 1) % num_cols + 1)
        elif isinstance(data, list) and isinstance(data[0], np.ndarray):
            # Plot list of tensors
            for j, tensor in enumerate(data, start=1):
                trace = go.Heatmap(
                    z=tensor,
                    colorscale="Viridis",
                    name=f"Tensor {j}",
                )
                fig.add_trace(trace, row=(i - 1) // num_cols + 1, col=(i - 1) % num_cols + 1)
        elif isinstance(data, list):
            # Plot list
            for j, item in enumerate(data, start=1):
                if isinstance(item, list):
                    # Plot list of lists
                    item_data = ", ".join(str(x) for x in item)
                    trace = go.Scatter(
                        x=[j],
                        y=[len(item)],
                        mode="lines+markers",
                        text=item_data,
                        hoverinfo="text",
                        name=f"List {j}",
                    )
                else:
                    # Plot list of non-list values
                    trace = go.Bar(
                        x=[j],
                        y=[item],
                        text=str(item),
                        textposition="auto",
                        name=f"Item {j}",
                    )
                fig.add_trace(trace, row=(i - 1) // num_cols + 1, col=(i - 1) % num_cols + 1)
        elif isinstance(data, np.ndarray):
            # Plot tensor
            trace = go.Heatmap(
                z=data,
                colorscale="Viridis",
                name="Tensor",
            )
            fig.add_trace(trace, row=(i - 1) // num_cols + 1, col=(i - 1) % num_cols + 1)
        else:
            # Plot single value
            trace = go.Bar(
                x=["Value"],
                y=[data],
                text=str(data),
                textposition="auto",
                name="Value",
            )
            fig.add_trace(trace, row=(i - 1) // num_cols + 1, col=(i - 1) % num_cols + 1)

    # Customize layout
    fig.update_layout(
        title="Statistical Analysis of Data Structures",
        height=num_rows * 400,
        showlegend=False,
    )

    # Display the plot
    fig.show()


# Example usage
data1 = {
    "Category 1": [
        np.random.rand(3, 3),
        np.random.rand(4, 4),
    ],
    "Category 2": [
        np.random.rand(2, 2),
        np.random.rand(5, 5),
    ],
}

data2 = [
    np.random.rand(3, 3),
    np.random.rand(4, 4),
    np.random.rand(2, 2),
]

data3 = np.random.rand(5, 5)

data4 = {
    "Fruit": "apple",
    "Color": "red",
    "Number": 42,
}

data5 = [1, 2, 3, 4, 5]

data6 = "Hello, World!"

# User can provide any combination and structure of data
plot_data_structures(data1, data2, data3)
plot_data_structures(data4, data5, data6)
plot_data_structures(data1, data4, data5)
plot_data_structures(data2, data3, data6)


In [15]:
from typing import List, Dict, Union, Any
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import numpy as np

def plot_data_structures(*data_structures: Any) -> None:
    """
    Plots various data structures using Plotly based on user input.

    Args:
        *data_structures: Variable number of data structures of any type.

    Returns:
        None
    """
    num_plots = len(data_structures)
    num_rows = (num_plots + 1) // 2
    num_cols = 2

    # Create subplots
    fig = make_subplots(rows=num_rows, cols=num_cols, subplot_titles=[f"Data {i+1}" for i in range(num_plots)])

    for i, data in enumerate(data_structures, start=1):
        if isinstance(data, dict):
            # Plot dictionary
            for key, value in data.items():
                if isinstance(value, list) and isinstance(value[0], np.ndarray):
                    # Plot list of tensors
                    for j, tensor in enumerate(value, start=1):
                        trace = go.Heatmap(
                            z=tensor,
                            colorscale="Viridis",
                            name=f"{key} - Tensor {j}",
                        )
                        fig.add_trace(trace, row=(i - 1) // num_cols + 1, col=(i - 1) % num_cols + 1)
                else:
                    trace = go.Scatter(
                        x=[str(key)],
                        y=[str(value)],
                        mode="markers",
                        marker=dict(size=10),
                        text=str(value),
                        hoverinfo="text",
                        name=str(key),
                    )
                    fig.add_trace(trace, row=(i - 1) // num_cols + 1, col=(i - 1) % num_cols + 1)
        elif isinstance(data, list):
            if isinstance(data[0], np.ndarray):
                # Plot list of tensors
                for j, tensor in enumerate(data, start=1):
                    trace = go.Heatmap(
                        z=tensor,
                        colorscale="Viridis",
                        name=f"Tensor {j}",
                    )
                    fig.add_trace(trace, row=(i - 1) // num_cols + 1, col=(i - 1) % num_cols + 1)
            else:
                # Plot list
                for j, item in enumerate(data, start=1):
                    trace = go.Scatter(
                        x=[j],
                        y=[str(item)],
                        mode="markers",
                        marker=dict(size=10),
                        text=str(item),
                        hoverinfo="text",
                        name=f"Item {j}",
                    )
                    fig.add_trace(trace, row=(i - 1) // num_cols + 1, col=(i - 1) % num_cols + 1)
        elif isinstance(data, np.ndarray):
            # Plot tensor
            trace = go.Heatmap(
                z=data,
                colorscale="Viridis",
                name="Tensor",
            )
            fig.add_trace(trace, row=(i - 1) // num_cols + 1, col=(i - 1) % num_cols + 1)
        else:
            # Plot single value
            trace = go.Scatter(
                x=["Value"],
                y=[str(data)],
                mode="markers",
                marker=dict(size=10),
                text=str(data),
                hoverinfo="text",
                name="Value",
            )
            fig.add_trace(trace, row=(i - 1) // num_cols + 1, col=(i - 1) % num_cols + 1)

    # Customize layout
    fig.update_layout(
        title="Statistical Analysis of Data Structures",
        height=num_rows * 400,
        showlegend=False,
    )

    # Display the plot
    fig.show()


# Example usage
data1 = {
    "Category 1": [
        np.random.rand(3, 3),
        np.random.rand(4, 4),
    ],
    "Category 2": [
        np.random.rand(2, 2),
        np.random.rand(5, 5),
    ],
}

data2 = [
    np.random.rand(3, 3),
    np.random.rand(4, 4),
]

data3 = np.random.rand(5, 5)

data4 = 42

data5 = {
    "Name": "John",
    "Age": 30,
    "City": "New York"
}

# User can provide any combination of data structures
plot_data_structures(data3)
plot_data_structures(data1, data3)
plot_data_structures(data2, data3)
plot_data_structures(data1, data2, data3)
plot_data_structures(data4)
plot_data_structures(data5)


In [20]:
from typing import Any, List, Dict, Union
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import numpy as np

def plot_data_structures(*data_structures: Any) -> None:
    """
    Plots various data structures using Plotly based on user input.

    Args:
        *data_structures: Variable number of data structures of any type.

    Returns:
        None
    """
    num_plots = len(data_structures)
    num_rows = (num_plots + 1) // 2
    num_cols = 2

    # Create subplots with modern layout
    fig = make_subplots(
        rows=num_rows,
        cols=num_cols,
        subplot_titles=[f"Data {i+1}" for i in range(num_plots)],
        horizontal_spacing=0.05,
        vertical_spacing=0.1,
    )

    for i, data in enumerate(data_structures, start=1):
        if isinstance(data, dict):
            # Plot dictionary
            for key, value in data.items():
                if isinstance(value, list) and isinstance(value[0], np.ndarray):
                    # Plot dictionary of lists of tensors
                    for j, tensor in enumerate(value, start=1):
                        trace = go.Heatmap(
                            z=tensor,
                            colorscale="Viridis",
                            name=f"{key} - Tensor {j}",
                            colorbar=dict(title="Value"),
                        )
                        fig.add_trace(trace, row=(i - 1) // num_cols + 1, col=(i - 1) % num_cols + 1)
                elif isinstance(value, list):
                    # Plot dictionary of lists
                    for j, item in enumerate(value, start=1):
                        if isinstance(item, list):
                            # Plot dictionary of lists of lists
                            item_data = ", ".join(str(x) for x in item)
                        else:
                            item_data = str(item)
                        trace = go.Scatter(
                            x=[j],
                            y=[key],
                            mode="markers",
                            marker=dict(
                                size=15,
                                color=j,
                                colorscale="Viridis",
                                showscale=True,
                                colorbar=dict(title="Index"),
                            ),
                            text=item_data,
                            hoverinfo="text",
                            name=key,
                        )
                        fig.add_trace(trace, row=(i - 1) // num_cols + 1, col=(i - 1) % num_cols + 1)
                else:
                    # Plot dictionary of non-list values
                    trace = go.Bar(
                        x=[key],
                        y=[value],
                        text=str(value),
                        textposition="auto",
                        name=key,
                        marker=dict(
                            color=value,
                            colorscale="Viridis",
                            showscale=True,
                            colorbar=dict(title="Value"),
                        ),
                    )
                    fig.add_trace(trace, row=(i - 1) // num_cols + 1, col=(i - 1) % num_cols + 1)
        elif isinstance(data, list) and isinstance(data[0], np.ndarray):
            # Plot list of tensors
            for j, tensor in enumerate(data, start=1):
                trace = go.Heatmap(
                    z=tensor,
                    colorscale="Viridis",
                    name=f"Tensor {j}",
                    colorbar=dict(title="Value"),
                )
                fig.add_trace(trace, row=(i - 1) // num_cols + 1, col=(i - 1) % num_cols + 1)
        elif isinstance(data, list):
            # Plot list
            for j, item in enumerate(data, start=1):
                if isinstance(item, list):
                    # Plot list of lists
                    item_data = ", ".join(str(x) for x in item)
                    trace = go.Scatter(
                        x=[j],
                        y=[len(item)],
                        mode="lines+markers",
                        text=item_data,
                        hoverinfo="text",
                        name=f"List {j}",
                        line=dict(
                            color=j,
                            colorscale="Viridis",
                            showscale=True,
                            colorbar=dict(title="Index"),
                        ),
                    )
                else:
                    # Plot list of non-list values
                    trace = go.Bar(
                        x=[j],
                        y=[item],
                        text=str(item),
                        textposition="auto",
                        name=f"Item {j}",
                        marker=dict(
                            color=item,
                            colorscale="Viridis",
                            showscale=True,
                            colorbar=dict(title="Value"),
                        ),
                    )
                fig.add_trace(trace, row=(i - 1) // num_cols + 1, col=(i - 1) % num_cols + 1)
        elif isinstance(data, np.ndarray):
            # Plot tensor
            trace = go.Heatmap(
                z=data,
                colorscale="Viridis",
                name="Tensor",
                colorbar=dict(title="Value"),
            )
            fig.add_trace(trace, row=(i - 1) // num_cols + 1, col=(i - 1) % num_cols + 1)
        else:
            # Plot single value
            trace = go.Bar(
                x=["Value"],
                y=[data],
                text=str(data),
                textposition="auto",
                name="Value",
                marker=dict(
                    color=data,
                    colorscale="Viridis",
                    showscale=True,
                    colorbar=dict(title="Value"),
                ),
            )
            fig.add_trace(trace, row=(i - 1) // num_cols + 1, col=(i - 1) % num_cols + 1)

    # Customize layout with modern style
    fig.update_layout(
        title=dict(
            text="Statistical Analysis of Data Structures",
            font=dict(size=24, color="black"),
            x=0.5,
            y=0.95,
        ),
        height=num_rows * 500,
        showlegend=False,
        template="plotly_white",
        hoverlabel=dict(font=dict(size=14)),
        margin=dict(l=50, r=50, t=100, b=50),
    )

    # Display the plot
    fig.show()


# Example usage
data1 = {
    "Category 1": [
        np.random.rand(3, 3),
        np.random.rand(4, 4),
    ],
    "Category 2": [
        np.random.rand(2, 2),
        np.random.rand(5, 5),
    ],
}

data2 = [
    np.random.rand(3, 3),
    np.random.rand(4, 4),
    np.random.rand(2, 2),
]

data3 = np.random.rand(5, 5)

data4 = {
    "Fruit": "apple",
    "Color": "red",
    "Number": 42,
}

data5 = [1, 2, 3, 4, 5]

data6 = "Hello, World!"

# User can provide any combination and structure of data
plot_data_structures(data1, data2, data3)
# plot_data_structures(data4, data5, data6)
# plot_data


In [19]:
from typing import Any, List, Dict, Union
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import numpy as np

def plot_data_structures(*data_structures: Any) -> None:
    """
    Plots various data structures using Plotly based on user input.

    Args:
        *data_structures: Variable number of data structures of any type.

    Returns:
        None
    """
    num_plots = len(data_structures)
    num_rows = (num_plots + 1) // 2
    num_cols = 2

    # Create subplots with modern layout
    fig = make_subplots(
        rows=num_rows,
        cols=num_cols,
        subplot_titles=[f"Data {i+1}" for i in range(num_plots)],
        horizontal_spacing=0.1,
        vertical_spacing=0.1,
    )

    for i, data in enumerate(data_structures, start=1):
        if isinstance(data, dict):
            # Plot dictionary
            for key, value in data.items():
                if isinstance(value, list) and isinstance(value[0], np.ndarray):
                    # Plot dictionary of lists of tensors
                    for j, tensor in enumerate(value, start=1):
                        trace = go.Heatmap(
                            z=tensor,
                            colorscale="Viridis",
                            name=f"{key} - Tensor {j}",
                            colorbar=dict(title="Value"),
                        )
                        fig.add_trace(trace, row=(i - 1) // num_cols + 1, col=(i - 1) % num_cols + 1)
                elif isinstance(value, list):
                    # Plot dictionary of lists
                    for j, item in enumerate(value, start=1):
                        if isinstance(item, list):
                            # Plot dictionary of lists of lists
                            item_data = ", ".join(str(x) for x in item)
                        else:
                            item_data = str(item)
                        trace = go.Scatter(
                            x=[j],
                            y=[key],
                            mode="markers",
                            marker=dict(
                                size=15,
                                color=j,
                                colorscale="Viridis",
                                showscale=True,
                                colorbar=dict(title="Index"),
                            ),
                            text=item_data,
                            hoverinfo="text",
                            name=key,
                        )
                        fig.add_trace(trace, row=(i - 1) // num_cols + 1, col=(i - 1) % num_cols + 1)
                else:
                    # Plot dictionary of non-list values
                    trace = go.Bar(
                        x=[key],
                        y=[value],
                        text=str(value),
                        textposition="auto",
                        name=key,
                        marker=dict(
                            color="rgba(0, 0, 255, 0.6)",
                            line=dict(color="rgba(0, 0, 255, 1.0)", width=2),
                        ),
                    )
                    fig.add_trace(trace, row=(i - 1) // num_cols + 1, col=(i - 1) % num_cols + 1)
        elif isinstance(data, list) and isinstance(data[0], np.ndarray):
            # Plot list of tensors
            for j, tensor in enumerate(data, start=1):
                trace = go.Heatmap(
                    z=tensor,
                    colorscale="Viridis",
                    name=f"Tensor {j}",
                    colorbar=dict(title="Value"),
                )
                fig.add_trace(trace, row=(i - 1) // num_cols + 1, col=(i - 1) % num_cols + 1)
        elif isinstance(data, list):
            # Plot list
            for j, item in enumerate(data, start=1):
                if isinstance(item, list):
                    # Plot list of lists
                    item_data = ", ".join(str(x) for x in item)
                    trace = go.Scatter(
                        x=[j],
                        y=[len(item)],
                        mode="lines+markers",
                        text=item_data,
                        hoverinfo="text",
                        name=f"List {j}",
                        line=dict(width=3),
                        marker=dict(
                            size=10,
                            color=j,
                            colorscale="Viridis",
                            showscale=True,
                            colorbar=dict(title="Index"),
                        ),
                    )
                else:
                    # Plot list of non-list values
                    trace = go.Bar(
                        x=[j],
                        y=[item],
                        text=str(item),
                        textposition="auto",
                        name=f"Item {j}",
                        marker=dict(
                            color="rgba(0, 255, 0, 0.6)",
                            line=dict(color="rgba(0, 255, 0, 1.0)", width=2),
                        ),
                    )
                fig.add_trace(trace, row=(i - 1) // num_cols + 1, col=(i - 1) % num_cols + 1)
        elif isinstance(data, np.ndarray):
            # Plot tensor
            trace = go.Heatmap(
                z=data,
                colorscale="Viridis",
                name="Tensor",
                colorbar=dict(title="Value"),
            )
            fig.add_trace(trace, row=(i - 1) // num_cols + 1, col=(i - 1) % num_cols + 1)
        else:
            # Plot single value
            trace = go.Bar(
                x=["Value"],
                y=[data],
                text=str(data),
                textposition="auto",
                name="Value",
                marker=dict(
                    color="rgba(255, 0, 0, 0.6)",
                    line=dict(color="rgba(255, 0, 0, 1.0)", width=2),
                ),
            )
            fig.add_trace(trace, row=(i - 1) // num_cols + 1, col=(i - 1) % num_cols + 1)

    # Customize layout
    fig.update_layout(
        title=dict(
            text="Statistical Analysis of Data Structures",
            font=dict(size=24, color="black"),
            x=0.5,
            y=0.95,
        ),
        height=num_rows * 500,
        margin=dict(l=50, r=50, t=100, b=50),
        paper_bgcolor="rgba(240, 240, 240, 0.95)",
        plot_bgcolor="rgba(255, 255, 255, 0.95)",
        showlegend=False,
    )

    # Display the plot
    fig.show()


# Example usage
data1 = {
    "Category 1": [
        np.random.rand(3, 3),
        np.random.rand(4, 4),
    ],
    "Category 2": [
        np.random.rand(2, 2),
        np.random.rand(5, 5),
    ],
}

data2 = [
    np.random.rand(3, 3),
    np.random.rand(4, 4),
    np.random.rand(2, 2),
]

data3 = np.random.rand(5, 5)

data4 = {
    "Fruit": "apple",
    "Color": "red",
    "Number": 42,
}

data5 = [1, 2, 3, 4, 5]

data6 = "Hello"

plot_data_structures(data1,data2,data3,data4,data5,data6)
